In [1]:
import numpy as np

import heapq, random
class Stack:
    "A container with a last-in-first-out (LIFO) queuing policy."
    def __init__(self):
        self.list = []

    def push(self,item):
        "Push 'item' onto the stack"
        self.list.append(item)

    def pop(self):
        "Pop the most recently pushed item from the stack"
        return self.list.pop()

    def is_empty(self):
        "Returns true if the stack is empty"
        return len(self.list) == 0
    
    def size(self):
        return len(self.list)
    

class Queue:
    "A container with a first-in-first-out (FIFO) queuing policy."
    def __init__(self):
        self.list = []

    def push(self,item):
        "Enqueue the 'item' into the queue"
        self.list.insert(0,item)

    def pop(self):
        """
          Dequeue the earliest enqueued item still in the queue. This
          operation removes the item from the queue.
        """
        return self.list.pop()

    def is_empty(self):
        "Returns true if the queue is empty"
        return len(self.list) == 0

    

class PriorityQueue:
    """
      Implements a priority queue data structure. Each inserted item
      has a priority associated with it and the client is usually interested
      in quick retrieval of the lowest-priority item in the queue. This
      data structure allows O(1) access to the lowest-priority item.
    """
    def  __init__(self):
        self.heap = []
        self.count = 0

    def push(self, item, priority):
        entry = (priority, self.count, item)
        heapq.heappush(self.heap, entry)
        self.count += 1

    def pop(self):
        (_, _, item) = heapq.heappop(self.heap)
        return item

    def is_empty(self):
        return len(self.heap) == 0

    def update(self, item, priority):
        # If item already in priority queue with higher priority, update its priority and rebuild the heap.
        # If item already in priority queue with equal or lower priority, do nothing.
        # If item not in priority queue, do the same thing as self.push.
        for index, (p, c, i) in enumerate(self.heap):
            if i == item:
                if p <= priority:
                    break
                del self.heap[index]
                self.heap.append((priority, c, item))
                heapq.heapify(self.heap)
                break
        else:
            self.push(item, priority)
            
    def size(self):
        return len(self.heap)

In [2]:
goal_state = np.arange(1,10).reshape((3,3))
start_state = np.array([1,2,3,4,5,6,7,9,8]).reshape(3,3)
print("goal state:\n",goal_state)
print("start state:\n", start_state)

goal state:
 [[1 2 3]
 [4 5 6]
 [7 8 9]]
start state:
 [[1 2 3]
 [4 5 6]
 [7 9 8]]


In [3]:
nine_row = np.where(start_state==9)[0][0]
nine_col = np.where(start_state==9)[1][0]
print(nine_row)
print(nine_col)

2
1


In [4]:
def get_successors(state):
    successors = []
    nine_row = np.where(state==9)[0][0]
    nine_col = np.where(state==9)[1][0]
    cost = 1
    if nine_row>0:
        action = 'U'
        next_state = state.copy()
        next_state[nine_row,nine_col] = state[nine_row-1,nine_col]
        next_state[nine_row-1,nine_col] = 9
        successors.append((next_state,action,cost))
    if nine_row<2:
        action = 'D'
        next_state = state.copy()
        next_state[nine_row,nine_col] = state[nine_row-1,nine_col]
        next_state[nine_row-1,nine_col] = 9
        successors.append((next_state,action,cost))
    if (nine_col>0):
        action = 'L'
        next_state = state.copy()
        next_state[nine_row,nine_col] = state[nine_row,nine_col-1]
        next_state[nine_row,nine_col-1] = 9
        successors.append((next_state,action,cost))
    if (nine_col<2):
        action = 'R'
        next_state = state.copy()
        next_state[nine_row,nine_col] = state[nine_row,nine_col+1]
        next_state[nine_row,nine_col+1] = 9
        successors.append((next_state,action,cost))
    return successors

In [5]:
def is_goal_state(state):
    return (state==goal_state).all()

## un-informed search

In [6]:
algorithm = "DFS"
if algorithm == "BFS":
    fringe = Queue() # which data structure is needed for BFS?
elif algorithm == "DFS": 
    fringe = Stack() # which one for BFS ?
explored = set() # empty set of explored nodes
path = [] # empty list of actions
fringe.push(( start_state , path )) # push (start state , path) to fringe
#start_state = np.array([1,2,3,4,5,6,7,9,8]).reshape(3,3) # let pick an easy to solve start state

print("start state: \n",start_state)

while not fringe.is_empty(): # while fringe is not empty
    state , path = fringe.pop() # pop a successor from fringe
    hashable_state = tuple(state.ravel()) # in python 2d arrays cannot be hashed, therefore cannot be added to set --> cast state to 1d tuple
    if is_goal_state(state):# check if current node is the goal node
        print("victory...")
        print(state)
        break
    if hashable_state not in explored: # if the node poped from fringed is NOT already explored
        explored.add( hashable_state ) # add it to explored set
        for next_state , action , cost in get_successors( state ): # and get its children : child node (child state), child action (action that leads to that node), child cost (cost of that action) 
            new_path = path + [ action ] # new_path = current path + new action --> we put [] around new_action to cast new_action to a list element
            fringe.push( ( next_state , new_path ) ) # tuple of child's state and path     
            
print('path:', path)

start state: 
 [[1 2 3]
 [4 5 6]
 [7 9 8]]
victory...
[[1 2 3]
 [4 5 6]
 [7 8 9]]
path: ['R']


## informed search

In [7]:
def heuristic(state):
    nine_row = np.where(state==9)[0][0]
    nine_col = np.where(state==9)[1][0]
    cost = (len(state)-1-nine_row)+(len(state)-1-nine_col)
    return cost

start_state = np.random.permutation(np.arange(1,10)).reshape((3,3))
print("start state: \n",start_state)

fringe = PriorityQueue()
explored = set() # empty set of explored nodes
path = [] # empty list of actions
g_state = 0 # what is g(start state) ?
h_state = 0 # what is h(start state) ?
f_state = g_state + h_state # f_state = g_state + h_state
fringe.push(( start_state , path , g_state ), f_state ) #   push ( (state , path ,g_state) , f_state ) to fringe. check the push function in util.priorityQueue

while not fringe.is_empty(): # while fringe is not empty
    state , path , g_state  = fringe.pop() # pop a successor from fringe
    hashable_state = tuple(state.ravel())
    h_state = heuristic(state) # what is ur heuristic
    f_state = g_state + h_state # f_state = g_state + h_state
    if is_goal_state(state):# check if current node is the goal node
        print("victory...")
        print(state)
        break
    if hashable_state not in explored: # if the node poped from fringed is NOT already explored
        explored.add(hashable_state) # add it to explored set
        for next_state , action , cost in get_successors(state) : # get its children -> next state,  next action , cost of action
            current_path = path + [ action ] # current path + new action
            g_next_state = h_state + cost   # g_next_state = cost till now + cost of new action
            h_next_state = heuristic( next_state ) # h_next_state (for USC u can use the nullHeuristic function, for A* u can use other heuristics)
            f_next_state = g_next_state + h_next_state # f_next_state = g_next_state + h_next_state
            fringe.push(( next_state , current_path , g_next_state ), f_next_state ) # check the push function in util.priorityQueue

print('path:', path)

start state: 
 [[9 6 1]
 [5 3 7]
 [8 2 4]]
victory...
[[1 2 3]
 [4 5 6]
 [7 8 9]]
path: ['D', 'R', 'R', 'U', 'U', 'D', 'L', 'U', 'R', 'U', 'D', 'L', 'L', 'U', 'R', 'R', 'U', 'D', 'L', 'L', 'U', 'U', 'D', 'U', 'R', 'R', 'U', 'D', 'L', 'L', 'U', 'R', 'R', 'U', 'D', 'L', 'U', 'R', 'U', 'D', 'L', 'U', 'U', 'D', 'R', 'U', 'U', 'D', 'L', 'U', 'R', 'U', 'D', 'L', 'U', 'R', 'U', 'D', 'U', 'U', 'D']


In [ ]:
ج)
با توجه به اینکه هر حرکت یک هزینه دارد بنابراین تفاوتی ندارند. در واقع صف اولویت برای یو سی اس به همان صف عادی تبدیل میشود زیرا هزینه همه یکسان است.

In [ ]:
د)
4^9!